In [1]:
import gc
import json
import optuna
import pandas as pd
import numpy as np
import xgboost as xgb

from pathlib import Path
from numerapi import NumerAPI
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

from helper import print_both, categorical_encoded

napi = NumerAPI()
current_round = napi.get_current_round()

In [ ]:
# Tournament data changes every week so we specify the round in their name. Training
# and validation data only change periodically, so no need to download them every time.
print('Downloading dataset files...')

Path("./v4").mkdir(parents=False, exist_ok=True)
napi.download_dataset("v4/train.parquet")
napi.download_dataset("v4/validation.parquet")
napi.download_dataset("v4/live.parquet", f"v4/live_{current_round}.parquet")
napi.download_dataset("v4/validation_example_preds.parquet")
napi.download_dataset("v4/features.json")

In [2]:
ERA_COL = "era"
TARGET_COL = "target_nomi_v4_20"
DATA_TYPE_COL = "data_type"
EXAMPLE_PREDS_COL = "example_preds"

print('Reading minimal training data')
with open("v4/features.json", "r") as f:
    feature_metadata = json.load(f)

features = feature_metadata["feature_sets"]["medium"]
read_columns = features + [ERA_COL, DATA_TYPE_COL, TARGET_COL]

Reading minimal training data


In [3]:
training_data = pd.read_parquet('v4/train.parquet', columns=read_columns)
#training_data = training_data.replace([np.inf, -np.inf], np.nan)
training_data = categorical_encoded(training_data)


validation_data = pd.read_parquet('v4/validation.parquet', columns=read_columns)
#validation_data = validation_data.replace([np.inf, -np.inf], np.nan)

In [8]:
training_data = categorical_encoded(training_data)

target = TARGET_COL

data = training_data.copy()
optuna_target = data[target]

Names of categorical columns :  ['era', 'data_type']


In [9]:
train_x, test_x, train_y, test_y = train_test_split(data, optuna_target, test_size=0.2, random_state=42)

In [15]:
file = open(f'{current_round}_report.txt', 'w', encoding="utf-8")

def objective(trial):
    param = {
            'tree_method':'hist',  #this parameter means using the GPU when training our model to speedup the training process
            'n_estimators': trial.suggest_categorical('n_estimators', [300, 700]),
            'objective': trial.suggest_categorical('objective', ['reg:squarederror']),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 0, 5),
            'gamma': trial.suggest_float('gamma', 0, 5),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.0, 1.0),
            'subsample': trial.suggest_float('subsample', 0.0, 1.0),
            'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1),
            'max_depth': trial.suggest_int('max_depth', 3, 5),
            'random_state': trial.suggest_categorical('random_state', [2020]),
            'min_child_weight': trial.suggest_int('min_child_weight', 10, 300),
            'eval_metric': trial.suggest_categorical('eval_metric', ['mae'])
            }

    
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'validation_0-mae')
    model = xgb.XGBRegressor(**param)  
    
    model.fit(train_x, train_y, eval_set=[(test_x, test_y)], early_stopping_rounds=20, callbacks=[pruning_callback])
    
    preds = model.predict(test_x)
    
    mae = mean_absolute_error(test_y, preds)
    
    return mae


In [16]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20, show_progress_bar=True)
print_both(file, 'Number of finished trials:', len(study.trials))
print_both(file, 'Best trial:', study.best_trial.params)

gc.collect()

[I 2022-11-28 10:58:59,172] A new study created in memory with name: no-name-0289c8f7-6519-473b-adc2-2813a8d6514b
c:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\optuna\progress_bar.py:49: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()
  0%|          | 0/20 [00:00<?, ?it/s]

[0]	validation_0-mae:0.14999
[1]	validation_0-mae:0.14409
[2]	validation_0-mae:0.13842
[3]	validation_0-mae:0.13851
[4]	validation_0-mae:0.13306
[5]	validation_0-mae:0.12783
[6]	validation_0-mae:0.12280
[7]	validation_0-mae:0.11798
[8]	validation_0-mae:0.11799
[9]	validation_0-mae:0.11336
[10]	validation_0-mae:0.10890
[11]	validation_0-mae:0.10462
[12]	validation_0-mae:0.10051
[13]	validation_0-mae:0.10056
[14]	validation_0-mae:0.09661
[15]	validation_0-mae:0.09282
[16]	validation_0-mae:0.08917
[17]	validation_0-mae:0.08922
[18]	validation_0-mae:0.08925
[19]	validation_0-mae:0.08929
[20]	validation_0-mae:0.08579
[21]	validation_0-mae:0.08243
[22]	validation_0-mae:0.07920
[23]	validation_0-mae:0.07610
[24]	validation_0-mae:0.07312
[25]	validation_0-mae:0.07314
[26]	validation_0-mae:0.07028
[27]	validation_0-mae:0.07030
[28]	validation_0-mae:0.07032
[29]	validation_0-mae:0.07035
[30]	validation_0-mae:0.07035
[31]	validation_0-mae:0.07037
[32]	validation_0-mae:0.06762
[33]	validation_0-ma

  5%|▌         | 1/20 [02:19<44:08, 139.37s/it]

[I 2022-11-28 11:01:18,560] Trial 0 finished with value: 0.0015295931370928884 and parameters: {'n_estimators': 700, 'objective': 'reg:squarederror', 'reg_lambda': 0.9458982362682626, 'reg_alpha': 0.6142755517627096, 'gamma': 1.5693147926644302, 'colsample_bytree': 0.6393106948732752, 'subsample': 0.6920886101698128, 'learning_rate': 0.03936921685016123, 'max_depth': 3, 'random_state': 2020, 'min_child_weight': 294, 'eval_metric': 'mae'}. Best is trial 0 with value: 0.0015295931370928884.


c:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
c:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


...

[0]	validation_0-mae:0.14267
[1]	validation_0-mae:0.13578
[2]	validation_0-mae:0.12923
[3]	validation_0-mae:0.12299
[4]	validation_0-mae:0.11706
[5]	validation_0-mae:0.11141
[6]	validation_0-mae:0.10603
[7]	validation_0-mae:0.10092
[8]	validation_0-mae:0.10092
[9]	validation_0-mae:0.09605
[10]	validation_0-mae:0.09141
[11]	validation_0-mae:0.08700
[12]	validation_0-mae:0.08280
[13]	validation_0-mae:0.07881
[14]	validation_0-mae:0.07500
[15]	validation_0-mae:0.07138
[16]	validation_0-mae:0.06794
[17]	validation_0-mae:0.06466
[18]	validation_0-mae:0.06471
[19]	validation_0-mae:0.06476
[20]	validation_0-mae:0.06164
[21]	validation_0-mae:0.05867
[22]	validation_0-mae:0.05585
[23]	validation_0-mae:0.05316
[24]	validation_0-mae:0.05060
[25]	validation_0-mae:0.05060
[26]	validation_0-mae:0.04816
[27]	validation_0-mae:0.04816
[28]	validation_0-mae:0.04584
[29]	validation_0-mae:0.04363
[30]	validation_0-mae:0.04363
[31]	validation_0-mae:0.04363
[32]	validation_0-mae:0.04153
[33]	validation_0-ma

100%|██████████| 20/20 [23:27<00:00, 70.39s/it]

[I 2022-11-28 11:22:26,919] Trial 19 pruned. Trial was pruned at iteration 33.


Number of finished trials: 20
Best trial: {'n_estimators': 300, 'objective': 'reg:squarederror', 'reg_lambda': 0.9600341204525804, 'reg_alpha': 0.07855811026900472, 'gamma': 1.0036481295357302, 'colsample_bytree': 0.6881113467654387, 'subsample': 0.8818520861043739, 'learning_rate': 0.0606575491282112, 'max_depth': 4, 'random_state': 2020, 'min_child_weight': 179, 'eval_metric': 'mae'}


6325

In [19]:
y

id
n000101811a8a843    0.50
n001e1318d5072ac    0.00
n002a9c5ab785cbb    0.50
n002ccf6d0e8c5ad    0.25
n0051ab821295c29    0.50
                    ... 
nffcc311c1b4ffb2     NaN
nffcca880507acfc     NaN
nffce3e57217e7a3     NaN
nffd5ec0564586e7     NaN
nffe284f043b586f     NaN
Name: target_nomi_v4_20, Length: 2366074, dtype: float32

In [18]:
gc.collect()

validation_data = categorical_encoded(validation_data)
    
X = validation_data.drop(target, axis=1)
y = validation_data[target]


if os.path.exists(f'best_trial_{current_round}.parquet'):
    trial = study.trials_dataframe()
    trial_best_score = trial['value'].min()

    historic_best_studies = pd.read_parquet(f'best_trial_{current_round}.parquet')
    historic_best_score = historic_best_studies['value'].min()

    if trial_best_score > historic_best_score:
        print_both(file, f'- Current best score: {trial_best_score} is no improvement over historic best score: {historic_best_score}, do nothing')
        
    else:
        print_both(file, f'- Current best score: {trial_best_score} is an improvement over historic best score: {historic_best_score}, update best trial')
        trial.to_parquet(f'best_trial_{current_round}.parquet')
        print_both(file, f'- Data saved to best_trial_{current_round}.parquet')
        
        best_trial = study.best_trial.params
        best_trial['tree_method'] = 'hist'

        model = xgb.XGBRegressor(**best_trial)
        model.fit(X, y)

        preds = model.predict(X)

        print_both(file, f"MAE for set: {mean_absolute_error(y, preds)}")
        print_both(file, f"R2 for set: {r2_score(y, preds)}")
        joblib.dump(model, f'{current_round}_predictor.joblib')
else:
    trial = study.trials_dataframe()
    trial.to_parquet(f'best_trial_{current_round}.parquet')
    best_trial = study.best_trial.params
    best_trial['tree_method'] = 'hist'

    model = xgb.XGBRegressor(**best_trial)
    model.fit(X, y)

    preds = model.predict(X)

    print_both(file, f"MAE for set: {mean_absolute_error(y, preds)}")
    print_both(file, f"R2 for set: {r2_score(y, preds)}")
    joblib.dump(model, f'{current_round}_predictor.joblib')

Names of categorical columns :  ['era', 'data_type']
- Current best score: 0.0009924035985022783 is an improvement over historic best score: 0.0009924035985022783, update best trial
- Data saved to best_trial_364.parquet


XGBoostError: [11:24:50] C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/data/data.cc:487: Check failed: valid: Label contains NaN, infinity or a value too large.

In [ ]:
 
read_columns = features + [ERA_COL, DATA_TYPE_COL, TARGET_COL]

# note: sometimes when trying to read the downloaded data you get an error about invalid magic parquet bytes...
# if so, delete the file and rerun the napi.download_dataset to fix the corrupted file
training_data = pd.read_parquet('v4/train.parquet',
                                columns=read_columns)
validation_data = pd.read_parquet('v4/validation.parquet',
                                  columns=read_columns)
live_data = pd.read_parquet(f'v4/live_{current_round}.parquet',
                                  columns=read_columns)